<a href="https://colab.research.google.com/github/gabrielagcam/Detec-o-precoce-de-AVCs/blob/main/RF/Trainning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/gabrielagcam/Detec-o-precoce-de-AVCs.git

fatal: destination path 'Detec-o-precoce-de-AVCs' already exists and is not an empty directory.


In [ ]:
# Instalação necessária
!pip install tensorflow_decision_forests
!pip install tqdm tqdm_joblib

import pandas as pd
import tensorflow_decision_forests as tfdf
import tensorflow as tf
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix

from tqdm import tqdm
from tqdm_joblib import tqdm_joblib

In [ ]:
# Carregar os dados tratados
df = pd.read_csv("Detec-o-precoce-de-AVCs/Data/Stroke.csv")

# Separar X e y
X = df.drop('stroke', axis=1)
y = df['stroke']

In [ ]:
# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [ ]:
# Pipeline com seleção de features e RandomForest
pipeline = Pipeline([
    ('kbest', SelectKBest(score_func=f_classif)), # implementando SelectKBest
    ('rf', RandomForestClassifier(random_state=42))
])

In [ ]:
# Definindo o grid de hiperparâmetros
param_grid = {
    'kbest__k': list(range(1, X_train.shape[1] + 1)),  # de 1 até todas as features
    'rf__n_estimators': [50],  # número de árvores
    'rf__max_depth': [None, 5, 10, 20],       # incluir também max_depth para refinar
    'rf__criterion': ['gini', 'entropy']      # diferentes critérios de divisão
}

# Configurando o GridSearchCV com validação cruzada de 5 folds
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=0
)

# Estimando o número total de iterações
total_iter = grid_search.cv * np.prod([len(v) for v in param_grid.values()])
print(f"Total de iterações estimadas: {total_iter}")

# Executando o grid search com barra de progresso
with tqdm_joblib(tqdm(desc="GridSearchCV", total=total_iter)) as progress_bar:
    grid_search.fit(X_train, y_train)

Total de iterações estimadas: 760




GridSearchCV:   0%|          | 0/760 [00:00<?, ?it/s]

  0%|          | 0/760 [00:00<?, ?it/s]

In [ ]:
# Resultados
print("\nMelhores hiperparâmetros encontrados:")
print(grid_search.best_params_)

print(f"\nMelhor acurácia média na validação cruzada: {grid_search.best_score_:.4f}")


Melhores hiperparâmetros encontrados:
{'kbest__k': 18, 'rf__criterion': 'gini', 'rf__max_depth': None, 'rf__n_estimators': 50}

Melhor acurácia média na validação cruzada: 0.9875


In [ ]:
# Avaliação final no conjunto de teste
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

In [ ]:
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred))
print("Matriz de Confusão:\n", confusion_matrix(y_test, y_pred))


Relatório de Classificação:
               precision    recall  f1-score   support

           0       1.00      0.98      0.99       951
           1       0.98      1.00      0.99       943

    accuracy                           0.99      1894
   macro avg       0.99      0.99      0.99      1894
weighted avg       0.99      0.99      0.99      1894

Matriz de Confusão:
 [[930  21]
 [  0 943]]
